# <span style="color:#0b486b">  FIT3181: Deep Learning (2022)</span>
***
*CE/Lecturer:* Dr **Trung Le** | trunglm@monash.edu <br/>
*Head Tutor:* Mr **Thanh Nguyen** | thanh.nguyen4@monash.edu <br/>
<br/>
Department of Data Science and AI, Faculty of Information Technology, Monash University, Australia
***

# <span style="color:#0b486b">  Student Information</span>
Surname: **Khong**  <br/>
Firstname: **Lap Hoe**    <br/>
Student ID: **32114818**    <br/>
Email: **lkho0007@student.monash.edu**    <br/>
Your tutorial time: **Friday 2PM**    <br/>
***

## <span style="color:#0b486b">Assignment 02: Neural Embedding and Sequence Modelling</span>
### Due: <span style="color:red">11:59pm 23 October 2022</span>  (Sunday)

#### <span style="color:red">Important note:</span> This is an **individual** assignment. It contributes **20%** to you final mark. Read the assignment instruction carefully.

## <span style="color:#0b486b">Instructions</span>

This notebook has been prepared for your to complete Assignment 2. The theme of this assignment is about practical machine learning knowledge and skills in deep neural networks, word embedding and text analytics. Some sections have been partially completed to help you get
started. **The total marks for this notebook is 100**.

* Before you start, read the entire notebook carefully once to understand what you need to do. <br><br>
* For each cell marked with **#YOU ARE REQUIRED TO INSERT YOUR CODES IN THIS CELL**, there will be places where you **must** supply your own codes when instructed. <br>

This assignment contains **five** parts:

* Part 1: Questions on downloading and preprocessing data **[5 points]**
* Part 2: Questions on using Word2Vect to transform texts to vectors **[20 points]**
* Part 3: Coding assessment on Text CNN for sequence modeling and neural embedding **[10 points]**
* Part 4: Coding assessment on BERT for a feature extraction **[10 points]**
* Part 5: Coding assessment on RNNs for sequence modeling and neural embedding **[55 points]**


**Hint**: This assignment was essentially designed based on the lectures and tutorials sessions covered from Weeks 7 to 10. You are strongly recommended to go through these contents thoroughly which might help you to complete this assignment.

## <span style="color:#0b486b">What to submit</span>

This assignment is to be completed individually and submitted to Moodle unit site. **By the due date, you are required to submit one  <span style="color:red; font-weight:bold">single zip file, named xxx_assignment02_solution.zip</span> where `xxx` is your student ID, to the corresponding Assignment (Dropbox) in Moodle**. 

***For example, if your student ID is <span style="color:red; font-weight:bold">12356</span>, then gather all of your assignment solution to folder, create a zip file named <span style="color:red; font-weight:bold">123456_assignment02_solution.zip</span> and submit this file.***

Within this zip folder, you **must** submit the following files:
1.	**Assignment02_solution.ipynb**:  this is your Python notebook solution source file.
1.	**Assignment02_output.html**: this is the output of your Python notebook solution *exported* in html format.
1.	Any **extra files or folder** needed to complete your assignment (e.g., images used in your answers).

## <span style="color:#0b486b">Set random seeds</span>

We start with importing tensorflow and numpy and setting random seeds for TF and numpy. You can use any seeds you prefer.

In [99]:
import numpy as np
import tensorflow as tf

tf.random.set_seed(6789)
np.random.seed(6789)

## <span style="color:#0b486b">Part 1: Download and preprocess the data</span>

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 5 points]<span></div>

The dataset we use for this assignment is a question classification dataset for which the train set consists of $5,500$ questions belonging to 6 coarse question categories including:
- abbreviation (ABBR), 
- entity (ENTY), 
- description (DESC), 
- human (HUM), 
- location (LOC) and 
- numeric (NUM).


Preprocessing data is an inital and important step in any machine learning or deep learning projects. The following *DataManager* class helps you to download data and preprocess data for the later questions of this assignment. 

In [100]:
import os
import zipfile
import collections
from six.moves import range
from six.moves.urllib.request import urlretrieve
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
plt.style.use('ggplot')

class DataManager:
    def __init__(self, verbose=True, maxlen= 50, random_state=6789):
        self.verbose = verbose
        self.max_sentence_len = 0
        self.str_questions = list()
        self.str_labels = list()
        self.numeral_labels = list()
        self.maxlen = maxlen
        self.numeral_data = list()
        self.random_state = random_state
        self.random = np.random.RandomState(random_state)
        
    @staticmethod
    def maybe_download(dir_name, file_name, url, verbose= True):
        if not os.path.exists(dir_name):
            os.mkdir(dir_name)
        if not os.path.exists(os.path.join(dir_name, file_name)):
            urlretrieve(url + file_name, os.path.join(dir_name, file_name))
        if verbose:
            print("Downloaded successfully {}".format(file_name))
    
    def read_data(self, dir_name, file_names):
        self.str_questions= list(); self.str_labels= list()
        for file_name in file_names:
            file_path= os.path.join(dir_name, file_name)
            with open(file_path, "r", encoding="latin-1") as f:
                for row in f:
                    row_str= row.split(":")
                    label, question= row_str[0], row_str[1]
                    question= question.lower()
                    self.str_labels.append(label)
                    self.str_questions.append(question[0:-1])
                    if self.max_sentence_len < len(self.str_questions[-1]):
                        self.max_sentence_len= len(self.str_questions[-1])
         
        # turns labels into numbers
        le= preprocessing.LabelEncoder()
        le.fit(self.str_labels)
        self.numeral_labels = np.array(le.transform(self.str_labels))
        self.str_classes= le.classes_
        self.num_classes= len(self.str_classes)
        if self.verbose:
            print("\nSample questions... \n")
            print(self.str_questions[0:5])
            print("Labels {}\n\n".format(self.str_classes))
    
    def manipulate_data(self):
        tokenizer = tf.keras.preprocessing.text.Tokenizer()
        tokenizer.fit_on_texts(self.str_questions)
        self.numeral_data = tokenizer.texts_to_sequences(self.str_questions)
        self.numeral_data = tf.keras.preprocessing.sequence.pad_sequences(self.numeral_data, padding='post', truncating= 'post', maxlen= self.maxlen)
        self.word2idx = tokenizer.word_index
        self.word2idx = {k:v for k,v in self.word2idx.items()}
        self.idx2word = {v:k for k,v in self.word2idx.items()}
        self.vocab_size = len(self.word2idx)
    
    def train_valid_split(self, train_ratio=0.9):
        idxs = np.random.permutation(np.arange(len(self.str_questions)))
        train_size = int(train_ratio*len(idxs)) +1
        self.train_str_questions, self.valid_str_questions = self.str_questions[0:train_size], self.str_questions[train_size:]
        self.train_numeral_data, self.valid_numeral_data = self.numeral_data[0:train_size], self.numeral_data[train_size:]
        self.train_numeral_labels, self.valid_numeral_labels = self.numeral_labels[0:train_size], self.numeral_labels[train_size:]
        self.tf_train_set = tf.data.Dataset.from_tensor_slices((self.train_numeral_data, self.train_numeral_labels))
        self.tf_valid_set = tf.data.Dataset.from_tensor_slices((self.valid_numeral_data, self.valid_numeral_labels))

In [101]:
print('Loading data...')
DataManager.maybe_download("Data", "train_1000.label", "http://cogcomp.org/Data/QA/QC/")
DataManager.maybe_download("Data", "TREC_10.label", "http://cogcomp.org/Data/QA/QC/")

dm = DataManager(maxlen=100)
dm.read_data("Data/", ["train_1000.label", "TREC_10.label"])   # read data
# If you want to play around with a bigger dataset, you can try train_set.label

Loading data...
Downloaded successfully train_1000.label
Downloaded successfully TREC_10.label

Sample questions... 

['manner how did serfdom develop in and then leave russia ?', 'cremat what films featured the character popeye doyle ?', "manner how can i find a list of celebrities ' real names ?", 'animal what fowl grabs the spotlight after the chinese year of the monkey ?', 'exp what is the full form of .com ?']
Labels ['ABBR' 'DESC' 'ENTY' 'HUM' 'LOC' 'NUM']




In [102]:
dm.manipulate_data()
dm.train_valid_split(train_ratio=0.8)

You now have a data manager, named *dm* containing the training and validiation sets in both text and numeric forms. Your task is to play around and read this code to figure out the meanings of some important attributes that will be used in the next parts.

#### <span style="color:red">**Question 1.1**</span> 
**What is the purpose of `self.train_str_questions` and `self.train_numeral_labels`? Write your code to print out the first five questions with labels in the training set.**

<div style="text-align: right"><span style="color:red">[1 point]</span></div> 

The purpose of `self.train_str_questions` is to store the actual string representation of the dataset. This was created by splitting the dataset into train test valid portions, and this represents the training portion. 

The purpose `self.train_numeral_labels` is to obtain the numeric representation of the labels. The original labels given were in text form. Having it as numeric categories can simplify the loss calculation process. 




In [103]:
#Your code here
five_questions = dm.train_str_questions[:5] 
five_labels_idx = dm.train_numeral_labels[:5] 
five_labels_text = dm.str_labels[:5]

for i in range(5):
    print(f"Question number {i + 1}: \n{five_questions[i]} \nwith index label {five_labels_idx[i]}, which is string label {five_labels_text[i]}\n")

Question number 1: 
manner how did serfdom develop in and then leave russia ? 
with index label 1, which is string label DESC

Question number 2: 
cremat what films featured the character popeye doyle ? 
with index label 2, which is string label ENTY

Question number 3: 
manner how can i find a list of celebrities ' real names ? 
with index label 1, which is string label DESC

Question number 4: 
animal what fowl grabs the spotlight after the chinese year of the monkey ? 
with index label 2, which is string label ENTY

Question number 5: 
exp what is the full form of .com ? 
with index label 0, which is string label ABBR



#### <span style="color:red">**Question 1.2**</span> 
**What is the purpose of `self.train_numeral_data`? Write your code to print out the first five questions in the numeric format with labels in the training set.**

<div style="text-align: right"><span style="color:red">[1 point]</span></div> 

The purpose of `self.train_numeral_data` is ... watch the lecture slides. 




In [104]:
#Your code here
#Your code here
five_questions = dm.train_numeral_data[:5] 
five_labels_idx = dm.train_numeral_labels[:5] 
five_labels_text = dm.str_labels[:5]

for i in range(5):
    print(f"Question number {i + 1}: \n{five_questions[i]} \nwith index label {five_labels_idx[i]}, which is string label {five_labels_text[i]}\n")

Question number 1: 
[  35   11   18 1030 1031    5   25  561 1032  562    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0] 
with index label 1, which is string label DESC

Question number 2: 
[  43    2  563 1033    1  164 1034 1035    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0  

#### <span style="color:red">**Question 1.3**</span> 
**What is the purpose of two dictionaries: `self.word2idx` and `self.idx2word`? Write your code to print out the first five key-value pairs of those dictionaries.**

<div style="text-align: right"><span style="color:red">[1 point]</span></div> 

The purpose of `self.word2idx` is ... 

the purpose of `self.idx2word` is ... 

In [105]:
for idx, key in enumerate(dm.word2idx):
    if idx == 5:
        break 
    else:
        print(f"Key-value pair {idx + 1}: Key is {key} and the value is {dm.word2idx[key]}") 

Key-value pair 1: Key is the and the value is 1
Key-value pair 2: Key is what and the value is 2
Key-value pair 3: Key is is and the value is 3
Key-value pair 4: Key is of and the value is 4
Key-value pair 5: Key is in and the value is 5


In [106]:
for idx, key in enumerate(dm.idx2word):
    if idx == 5:
        break 
    else:
        print(f"Key-value pair {idx + 1}: Key is {key} and the value is {dm.idx2word[key]}") 


Key-value pair 1: Key is 1 and the value is the
Key-value pair 2: Key is 2 and the value is what
Key-value pair 3: Key is 3 and the value is is
Key-value pair 4: Key is 4 and the value is of
Key-value pair 5: Key is 5 and the value is in


#### <span style="color:red">**Question 1.4**</span> 
**What is the purpose of `self.tf_train_set`? Write your code to print out the first five items of `self.tf_train_set`.**

<div style="text-align: right"><span style="color:red">[1 point]</span></div> 

The purpose of `self.tf_train_set` is ... 

In [138]:
#Your code here
tf_train_iter = dm.tf_train_set.as_numpy_iterator() 
for idx, (train_x, train_y) in enumerate(tf_train_iter):
    if idx == 5:
        break
    print(train_x.shape, train_y)

(100,) 1
(100,) 2
(100,) 1
(100,) 2
(100,) 0


#### <span style="color:red">**Question 1.5**</span> 
**What is the purpose of `self.tf_valid_set`? Write your code to print out the first five items of `self.tf_valid_set`.**

<div style="text-align: right"><span style="color:red">[1 point]</span></div> 

#Your answer here



In [108]:
#Your code here
tf_valid_iter = dm.tf_valid_set.as_numpy_iterator() 
for idx, (valid_x, valid_y) in enumerate(tf_valid_iter):
    if idx == 5:
        break
    print(valid_x, valid_y)

[  23    2   23   13  243    3   53    1 3031   20    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0] 4
[   6   12   98    1 3032    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0   

## <span style="color:#0b486b">Part 2: Using Word2Vect to transform texts to vectors </span>

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 20 points]<span></div>

In this part, you will be assessed on how to use a pretrained Word2Vect model for realizing a machine learning task. Basically, you will use this pretrained Word2Vect to transform the questions in the above dataset stored in the *data manager object dm* to numeric form for training a Support Vector Machine in sckit-learn.  

In [109]:
! pip install gensim


[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [110]:
import gensim.downloader as api
from gensim.models import Word2Vec
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

#### <span style="color:red">**Question 2.1**</span> 
**Write code to download the pretrained model *glove-wiki-gigaword-100*. Note that this model transforms a word in its dictionary to a $100$ dimensional vector.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div> 

In [111]:
word2vect = api.load('glove-wiki-gigaword-100')

#### <span style="color:red">**Question 2.2**</span> 

**Write code for the function *get_word_vector(word, model)* used to transform a word to a vector using the pretrained Word2Vect model *model*. Note that for a word not in the vocabulary of our *word2vect*, you need to return a vector $0$ with 100 dimensions.**

<div style="text-align: right"><span style="color:red">[3 points]</span></div> 

In [112]:
def get_word_vector(word, model):
    try:
        vector = model.get_vector(word)
    except: #word not in the vocabulary
        vector = np.zeros(100, dtype=np.float32)
    return vector

#### <span style="color:red">**Question 2.3**</span> 

**Write the code for the function `get_sentence_vector(sentence, important_score=None, model= None)`. Note that this function will transform a sentence to a 100-dimensional vector using the pretrained model *model*. In addition, the list *important_score* which has the same length as the *sentence* specifies the important scores of the words in the sentence. In your code, you first need to apply *softmax* function over *important_score* to obtain the important weight *important_weight* which forms a probability over the words of the sentence. Furthermore, the final vector of the sentence will be weighted sum of the individual vectors for words and the weights in *important_weight*.**
- $important\_weight = softmax(important\_score)$.
- $final\_vector= important\_weight[1]\times v[1] + important\_weight[2]\times v[2] + ...+ important\_weight[L]\times v[L]$ where $L$ is the length of the sentence and $v[i]$ is the vector representation of the $i-th$  word in this sentence.

**Note that if `important_score=None` is set by default, your function should return the average of all representation vectors corresponding to set `important_score=[1,1,...,1]`.**

<div style="text-align: right"><span style="color:red">[5 points]</span></div> 

In [113]:
def get_sentence_vector(sentence: str, important_score=None, model= None):
    sentence_list = sentence.split(" ") 
    sentence_vect = np.array([get_word_vector(word, model) for word in sentence_list])
    important_weight = np.array([[1] for word in sentence_list])
    
    if important_score is not None:
        important_weight = tf.nn.softmax(important_score) 
    return np.sum(sentence_vect * important_weight, axis=0)
    

#### <span style="color:red">**Question 2.4**</span> 

**Write code to transform the training questions in *dm.train_str_questions* to feature vectors. Note that after running the following cell, you must have $X\_train$ which is an numpy array of the feature vectors and $y\_train$ which is an array of numeric labels (*Hint: dm.train_numeral_labels*). You can add more lines to the following cell if necessary. In addition, you should decide the *important_score* by yourself. For example, you might reckon that the 1st score is 1, the 2nd score is decayed by 0.9, the 3rd is decayed by 0.9, and so on.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div> 

In [114]:
def generate_importance(init, sentence_list, decay_rate=0.9, model=None):
    return [[init * (decay_rate**i)] for word in sentence_list]

In [115]:
print("Transform training set to feature vectors...")

X_train= [get_sentence_vector(ques, important_score=generate_importance(1, ques.split(), model=word2vect), model=word2vect) for ques in dm.train_str_questions]
y_train= dm.train_numeral_labels

Transform training set to feature vectors...


In [116]:
# Simple assert that the final output of X_train is exactly the dimensions that we want
# (1201, 100), which is (num_sentences, feature_vec_len)
assert(np.shape(X_train) == (len(dm.train_str_questions), 100))

#### <span style="color:red">**Question 2.5**</span> 

**Write code to transform the training questions in *dm.valid_str_questions* to feature vectors. Note that after running the following cell, you must have $X\_valid$ which is an numpy array of the feature vectors and $y\_valid$ which is an array of numeric labels (*Hint: dm.valid_numeral_labels*). You can add more lines to the following cell if necessary. In addition, you should decide the *important_score* by yourself. For example, you might reckon that the 1st score is 1, the 2nd score is decayed by 0.9, the 3rd is decayed by 0.9, and so on.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div> 

In [117]:
print("Transform valid set to feature vectors...")
X_valid= [get_sentence_vector(ques, important_score=generate_importance(1, ques.split(), model=word2vect), model=word2vect) for ques in dm.valid_str_questions]
y_valid= dm.valid_numeral_labels

Transform valid set to feature vectors...


In [118]:
# Simple assert that the final output of X_valid is exactly the dimensions that we want
# (1201, 100), which is (num_sentences, feature_vec_len)
assert(np.shape(X_valid) == (len(dm.valid_str_questions), 100))

#### <span style="color:red">**Question 2.6**</span> 

**It is now to use *MinMaxScaler(feature_range=(-1,1))* in sckit-learn to scale both training and valid sets to the range $(-1,1)$.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div> 

In [119]:
scaler = MinMaxScaler(feature_range=(-1,1))
scaler.fit(X_train)
X_train = scaler.transform(X_train)
scaler.fit(X_valid)
X_valid = scaler.transform(X_valid)

#### <span style="color:red">**Question 2.7**</span> 

**Declare a support vector machine (the class *SVC*  in sckit-learn) with RBF kernel, $C=1$, $gamma= 2^{-3}$ and fit on the training set.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div> 

In [120]:
svm = SVC(C=1, kernel="rbf", gamma=0.0002)
svm.fit(X_train, y_train)

SVC(C=1, gamma=0.0002)

#### <span style="color:red">**Question 2.8**</span> 

**Finally, we use the trained *svm* to evaluate on the valid set $X\_valid$.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div> 

In [121]:
y_valid_pred= svm.predict(X_valid)
acc = svm.score(X_valid, y_valid)
print(acc)

0.2140468227424749


In [122]:
y_valid_pred

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

## <span style="color:#0b486b">Part 3: Text CNN for sequence modeling and neural embedding </span>

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 10 points]<span></div>

#### <span style="color:red">**Question 3.1**</span> 

**In what follows, you are required to complete the code for Text CNN for sentence classification. The paper of Text CNN can be found at this [link](https://www.aclweb.org/anthology/D14-1181.pdf). Here is the description of the Text CNN that you need to construct.**
- There are three attributes (properties or instance variables): *embed_size, state_size, data_manager*.
  - `embed_size`: the dimension of the vector space for which the words are embedded to using the embedding matrix.
  - `state_size`: the number of filters used in *Conv1D* (reference [here](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv1D)).
  - `data_manager`: the data manager to store information of the dataset.
- The detail of the computational process is as follows:
  - Given input $x$, we embed $x$ using the embedding matrix to obtain an $3D$ tensor $[batch\_size \times vocab\_size \times embed\_size]$ as $h$.
  - We feed $h$ to three Convd 1D layers, each of which has $state\_size$ filters, padding=same, activation= relu, and $kernel\_size= 3, 5, 7$ respectively to obtain $h1, h2, h3$. Note that each $h1, h2, h3$ is a 3D tensor with the shape $[batch\_size \times output\_size \times state\_size]$.
  - We then apply *GlobalMaxPool1D()* (reference [here](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GlobalMaxPool1D)) over $h1, h2, h3$ to obtain 2D tensors stored in $h1, h2, h3$ again.
  - We then concatenate three 2D tensors $h1, h2, h3$ to obtain $h$. Note that you need to specify the axis to concatenate.
  - We finally build up one dense layer on the top of $h$ for classification.
  
  <div style="text-align: right"><span style="color:red">[8 points]</span></div>
  

In [123]:
class TextCNN:
    def __init__(self, embed_size= 128, state_size=16, data_manager=None):
        self.data_manager = data_manager
        self.embed_size = embed_size
        self.state_size = state_size
    
    def build(self):
        x = tf.keras.layers.Input(shape=[None])
        h = tf.keras.layers.Embedding(self.data_manager.vocab_size +1, self.embed_size)(x)
        h1 = tf.keras.layers.Conv1D(filters=self.state_size, kernel_size=3, padding="same", activation="relu")(h)
        h2 = tf.keras.layers.Conv1D(filters=self.state_size, kernel_size=5, padding="same", activation="relu")(h)
        h3 = tf.keras.layers.Conv1D(filters=self.state_size, kernel_size=7, padding="same", activation="relu")(h)
        h1 = tf.keras.layers.GlobalMaxPool1D()(h1) 
        h2 = tf.keras.layers.GlobalMaxPool1D()(h2)
        h3 = tf.keras.layers.GlobalMaxPool1D()(h3)
        h = tf.concat([h1, h2, h3], 1)
        h = tf.keras.layers.Dense(self.data_manager.num_classes, activation='softmax')(h)
        self.model = tf.keras.Model(inputs=x, outputs=h) 
    
    def compile_model(self, *args, **kwargs):
        self.model.compile(*args, **kwargs)
    
    def fit(self, *args, **kwargs):
        return self.model.fit(*args, **kwargs)
    
    def evaluate(self, *args, **kwargs):
        self.model.evaluate(*args, **kwargs)


#### <span style="color:red">**Question 3.2**</span> 
**Here is the code to test TextCNN above. You can observe that TextCNN outperforms the traditional approach SVM + Word2Vect for this task. The reason is that TextCNN enables us to automatically learn the feature that fits to the task. This makes deep learning different from hand-crafted feature approaches. Complete the code to test the model. Note that when compiling the model, you can use the Adam optimizer.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div>

In [124]:
text_cnn = TextCNN(data_manager=dm)
text_cnn.build()
#Insert code here to compile the model
text_cnn.compile_model(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
#Insert code here to train the model on 20 epochs
text_cnn.fit(dm.tf_train_set.batch(50), epochs=20)

Epoch 1/20
25/25 [==============================] - 1s 35ms/step - loss: 1.6959 - accuracy: 0.4929
Epoch 2/20
25/25 [==============================] - 1s 35ms/step - loss: 1.3458 - accuracy: 0.8043
Epoch 3/20
25/25 [==============================] - 1s 35ms/step - loss: 0.8557 - accuracy: 0.9001
Epoch 4/20
25/25 [==============================] - 1s 36ms/step - loss: 0.4276 - accuracy: 0.9409
Epoch 5/20
25/25 [==============================] - 1s 35ms/step - loss: 0.2242 - accuracy: 0.9625
Epoch 6/20
25/25 [==============================] - 1s 36ms/step - loss: 0.1374 - accuracy: 0.9734
Epoch 7/20
25/25 [==============================] - 1s 36ms/step - loss: 0.0893 - accuracy: 0.9858
Epoch 8/20
25/25 [==============================] - 1s 39ms/step - loss: 0.0591 - accuracy: 0.9958
Epoch 9/20
25/25 [==============================] - 1s 38ms/step - loss: 0.0401 - accuracy: 0.9992
Epoch 10/20
25/25 [==============================] - 1s 39ms/step - loss: 0.0281 - accuracy: 1.0000
Epoch 11/

## <span style="color:#0b486b">Part 4: Sentence representation with BERT </span>

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 10 points]<span></div>

#### <span style="color:red">**Question 4.1**</span>
**Use a pretrained BERT model to extract feaure vectors for the training and valid sets. You should choose an appropriate pretrained BERT model that fits your computational resource.**
<div style="text-align: right"><span style="color:red">[6 points]</span></div>

In [125]:
#Insert your code for extracting training set


In [126]:
#Insert your code for extracting training set


#### <span style="color:red">**Question 4.2**</span>
**Use Logistic Regression to train on the training set and then evaluate on the valid set.**
<div style="text-align: right"><span style="color:red">[4 points]</span></div>

In [127]:
#Insert your code for training here


In [128]:
#Insert your code for testing here


## <span style="color:#0b486b">Part 5: RNNs for sequence modeling and neural embedding </span>

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 55 points]<span></div>

### <span style="color:#0b486b">5.1. One-directional RNNs for sequence modeling and neural embedding </span> ###

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 10 points]<span></div>

#### <span style="color:red">**Question 5.1.1**</span> 
**In this part, you need to construct an RNN to learn from the dataset of interest. Basically, you are required first to construct the class UniRNN (Uni-directional RNN) with the following requirements:**
- Attribute `data_manager (self.data_manager)`: specifies the data manager used to store data for the model.
- Attribute `cell_type (self.cell_type)`: can receive three values including `basic_rnn`, `gru`, and `lstm` which specifies the memory cells formed a hidden layer.
- `state_sizes (self.state_sizes)` indicates the list of the hidden sizes  of memory cells. For example, $state\_sizes = [64, 128]$ means that you have two hidden layers in your network with hidden sizes of $64$ and $128$ respectively.

**Note that when declaring an embedding layer for the network, you need to set *mask_zero=True* so that the padding zeros in the sentences will be masked and ignored. This helps to have variable length RNNs. For more detail, you can refer to this [link](https://www.tensorflow.org/guide/keras/masking_and_padding).**

<div style="text-align: right"><span style="color:red">[7 points]</span></div>

In [143]:
class UniRNN:
    def __init__(self, cell_type='gru', embed_size=128, state_sizes=[128, 64], data_manager=None):
        self.cell_type = cell_type
        self.state_sizes = state_sizes
        self.embed_size = embed_size
        self.data_manager = data_manager
        self.vocab_size = self.data_manager.vocab_size + 1
        self.embed_matrix = np.zeros((self.vocab_size, self.embed_size))

    # return the correspoding memory cell
    @staticmethod
    def get_layer(cell_type='gru', state_size=128, return_sequences=True, activation='tanh'):
        if cell_type == 'gru':
            return tf.keras.layers.GRU(state_size, return_sequences=return_sequences, activation=activation)
        elif cell_type == 'lstm':
            return tf.keras.layers.LSTM(state_size, return_sequences=return_sequences, activation=activation)
        else:
            return tf.keras.layers.RNN(
                tf.keras.layers.SimpleRNNCell(state_size, activation=activation),
                return_sequences=return_sequences)

    def build(self):
        x = tf.keras.layers.Input(shape=[None])
        h = tf.keras.layers.Embedding(
            self.vocab_size, self.embed_size, mask_zero=True)(x)
        num_layers = len(self.state_sizes)
        for i in range(num_layers):
            if i == num_layers - 1:
                h = self.get_layer(cell_type=self.cell_type,
                               state_size=self.state_sizes[i], return_sequences=False)(h)
            else:
                h = self.get_layer(cell_type=self.cell_type,
                                state_size=self.state_sizes[i])(h)
        h = tf.keras.layers.Dense(self.data_manager.num_classes, activation='softmax')(h)
        self.model = tf.keras.Model(inputs=x, outputs=h)

    def compile_model(self, *args, **kwargs):
        self.model.compile(*args, **kwargs)

    def fit(self, *args, **kwargs):
        return self.model.fit(*args, **kwargs)

    def evaluate(self, *args, **kwargs):
        self.model.evaluate(*args, **kwargs)


#### <span style="color:red">**Question 5.1.2**</span> 
**Run with basic RNN ('basic_rnn') cell with $embed\_size= 128, state\_sizes= [64, 128], data\_manager= dm$.**

<div style="text-align: right"><span style="color:red">[1 points]</span></div>

In [145]:
uni_rnn = UniRNN('basic_rnn', embed_size=128, state_sizes=[64,128], data_manager=dm)
uni_rnn.build()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
uni_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
uni_rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

Epoch 1/20
19/19 [==============================] - 2s 51ms/step - loss: 1.1063 - accuracy: 0.5737 - val_loss: 0.6174 - val_accuracy: 0.8595
Epoch 2/20
19/19 [==============================] - 1s 40ms/step - loss: 0.3714 - accuracy: 0.8793 - val_loss: 0.3352 - val_accuracy: 0.9197
Epoch 3/20
19/19 [==============================] - 1s 41ms/step - loss: 0.1318 - accuracy: 0.9675 - val_loss: 0.3151 - val_accuracy: 0.9298
Epoch 4/20
19/19 [==============================] - 1s 42ms/step - loss: 0.0568 - accuracy: 0.9850 - val_loss: 0.2494 - val_accuracy: 0.9465
Epoch 5/20
19/19 [==============================] - 1s 42ms/step - loss: 0.0353 - accuracy: 0.9917 - val_loss: 0.4082 - val_accuracy: 0.9365
Epoch 6/20
19/19 [==============================] - 1s 42ms/step - loss: 0.0484 - accuracy: 0.9833 - val_loss: 0.1527 - val_accuracy: 0.9498
Epoch 7/20
19/19 [==============================] - 1s 42ms/step - loss: 0.0226 - accuracy: 0.9950 - val_loss: 0.0936 - val_accuracy: 0.9632
Epoch 8/20
19

#### <span style="color:red">**Question 5.1.3**</span> 
**Run with GRU ('gru') cell with $embed\_size= 128, state\_sizes= [64, 128], data\_manager= dm$.**

<div style="text-align: right"><span style="color:red">[1 points]</span></div>

In [146]:
uni_rnn = UniRNN('gru', embed_size=128, state_sizes=[64,128], data_manager=dm)
uni_rnn.build()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
uni_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
uni_rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

Epoch 1/20
19/19 [==============================] - 7s 189ms/step - loss: 1.5639 - accuracy: 0.3514 - val_loss: 0.9807 - val_accuracy: 0.6823
Epoch 2/20
19/19 [==============================] - 2s 115ms/step - loss: 0.7324 - accuracy: 0.7186 - val_loss: 0.4318 - val_accuracy: 0.9030
Epoch 3/20
19/19 [==============================] - 2s 117ms/step - loss: 0.2791 - accuracy: 0.9284 - val_loss: 0.2193 - val_accuracy: 0.9398
Epoch 4/20
19/19 [==============================] - 2s 115ms/step - loss: 0.1065 - accuracy: 0.9750 - val_loss: 0.1828 - val_accuracy: 0.9431
Epoch 5/20
19/19 [==============================] - 2s 115ms/step - loss: 0.0842 - accuracy: 0.9784 - val_loss: 0.1931 - val_accuracy: 0.9398
Epoch 6/20
19/19 [==============================] - 2s 110ms/step - loss: 0.0528 - accuracy: 0.9867 - val_loss: 0.1322 - val_accuracy: 0.9599
Epoch 7/20
19/19 [==============================] - 2s 118ms/step - loss: 0.0186 - accuracy: 0.9967 - val_loss: 0.1436 - val_accuracy: 0.9599
Epoch 

#### <span style="color:red">**Question 5.1.4**</span> 
**Run with LSTM ('lstm') cell with $embed\_size= 128, state\_sizes= [64, 128], data\_manager= dm$.**

<div style="text-align: right"><span style="color:red">[1 points]</span></div>

In [147]:
uni_rnn = UniRNN('lstm', embed_size=128, state_sizes=[64,128], data_manager=dm)
uni_rnn.build()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
uni_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
uni_rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

Epoch 1/20
19/19 [==============================] - 8s 221ms/step - loss: 1.5089 - accuracy: 0.4480 - val_loss: 1.0912 - val_accuracy: 0.7592
Epoch 2/20
19/19 [==============================] - 2s 128ms/step - loss: 0.5596 - accuracy: 0.8293 - val_loss: 0.5256 - val_accuracy: 0.8963
Epoch 3/20
19/19 [==============================] - 2s 127ms/step - loss: 0.2152 - accuracy: 0.9467 - val_loss: 0.2901 - val_accuracy: 0.9130
Epoch 4/20
19/19 [==============================] - 2s 129ms/step - loss: 0.1149 - accuracy: 0.9734 - val_loss: 0.1596 - val_accuracy: 0.9498
Epoch 5/20
19/19 [==============================] - 2s 128ms/step - loss: 0.0624 - accuracy: 0.9867 - val_loss: 0.1536 - val_accuracy: 0.9498
Epoch 6/20
19/19 [==============================] - 2s 129ms/step - loss: 0.0393 - accuracy: 0.9900 - val_loss: 0.1842 - val_accuracy: 0.9465
Epoch 7/20
19/19 [==============================] - 2s 128ms/step - loss: 0.0158 - accuracy: 0.9967 - val_loss: 0.2138 - val_accuracy: 0.9465
Epoch 

### <span style="color:#0b486b">5.2. Bi-directional RNNs for sequence modeling and neural embedding </span> ###

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 10 points]<span></div>

#### <span style="color:red">**Question 5.2.1**</span> 
**In what follow, you will investigate BiRNN. The task is similar to Part 5.1 but you need to write the code for an BiRNN. Note that the function *get_layer(cell_type= 'gru', state_size= 128, return_sequences= False, activation = 'tanh')* has to return the hidden layer with bidirectional memory cells (e.g., Basic RNN, GRU, and LSTM cells).**

**Complete the code of the class *BiRNN*. Note that for the embedding layer you need to set *mask_zero=True*.**

<div style="text-align: right"><span style="color:red">[7 points]</span></div>

In [149]:
class BiRNN:
    def __init__(self, cell_type= 'gru', embed_size= 128, state_sizes= [128, 64], data_manager= None):
        self.cell_type = cell_type
        self.state_sizes = state_sizes
        self.embed_size = embed_size
        self.data_manager = data_manager
        self.vocab_size = self.data_manager.vocab_size +1
        
    # return the correspoding memory cell
    @staticmethod
    def get_layer(cell_type='gru', state_size=128, return_sequences=True, activation='tanh'):
        if cell_type == 'gru':
            return tf.keras.layers.Bidirectional(tf.keras.layers.GRU(state_size, return_sequences=return_sequences, activation=activation))
        elif cell_type == 'lstm':
            return tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(state_size, return_sequences=return_sequences, activation=activation))
        else:
            return tf.keras.layers.Bidirectional(tf.keras.layers.RNN(
                tf.keras.layers.SimpleRNNCell(state_size, activation=activation),
                return_sequences=return_sequences))
    
    def build(self):
        x = tf.keras.layers.Input(shape=[None])
        h = tf.keras.layers.Embedding(
            self.vocab_size, self.embed_size, mask_zero=True)(x)
        num_layers = len(self.state_sizes)
        for i in range(num_layers):
            if i == num_layers - 1:
                h = self.get_layer(cell_type=self.cell_type,
                               state_size=self.state_sizes[i], return_sequences=False)(h)
            else:
                h = self.get_layer(cell_type=self.cell_type,
                                state_size=self.state_sizes[i])(h)
        h = tf.keras.layers.Dense(dm.num_classes, activation='softmax')(h)
        self.model = tf.keras.Model(inputs=x, outputs=h)
        
    
    def compile_model(self, *args, **kwargs):
        self.model.compile(*args, **kwargs)
    
    def fit(self, *args, **kwargs):
        return self.model.fit(*args, **kwargs)
    
    def evaluate(self, *args, **kwargs):
        self.model.evaluate(*args, **kwargs)       


#### <span style="color:red">**Question 5.2.2**</span> 
**Run BiRNN for basic RNN ('basic_rnn') cell with $embed\_size= 128, state\_sizes= [64, 128], data\_manager= dm$.**

<div style="text-align: right"><span style="color:red">[1 points]</span></div>

In [150]:
bi_rnn = BiRNN('basic_rnn', embed_size=128, state_sizes=[64,128], data_manager=dm)
bi_rnn.build()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
bi_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
bi_rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

Epoch 1/20
19/19 [==============================] - 4s 101ms/step - loss: 0.7146 - accuracy: 0.7752 - val_loss: 0.2792 - val_accuracy: 0.9298
Epoch 2/20
19/19 [==============================] - 2s 83ms/step - loss: 0.0869 - accuracy: 0.9767 - val_loss: 0.3460 - val_accuracy: 0.9264
Epoch 3/20
19/19 [==============================] - 2s 87ms/step - loss: 0.0268 - accuracy: 0.9950 - val_loss: 0.1169 - val_accuracy: 0.9532
Epoch 4/20
19/19 [==============================] - 2s 88ms/step - loss: 0.0039 - accuracy: 0.9992 - val_loss: 0.1137 - val_accuracy: 0.9565
Epoch 5/20
19/19 [==============================] - 2s 89ms/step - loss: 9.5775e-04 - accuracy: 1.0000 - val_loss: 0.1307 - val_accuracy: 0.9532
Epoch 6/20
19/19 [==============================] - 2s 89ms/step - loss: 2.9662e-04 - accuracy: 1.0000 - val_loss: 0.1515 - val_accuracy: 0.9565
Epoch 7/20
19/19 [==============================] - 2s 90ms/step - loss: 1.0109e-04 - accuracy: 1.0000 - val_loss: 0.1769 - val_accuracy: 0.9565


#### <span style="color:red">**Question 5.2.3**</span> 
**Run BiRNN for GRU ('gru') cell with $embed\_size= 128, state\_sizes= [64, 128], data\_manager= dm$.**

<div style="text-align: right"><span style="color:red">[1 points]</span></div>

In [151]:
bi_rnn = BiRNN('gru', embed_size=128, state_sizes=[64,128], data_manager=dm)
bi_rnn.build()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
bi_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
bi_rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

Epoch 1/20
19/19 [==============================] - 18s 571ms/step - loss: 1.3020 - accuracy: 0.5520 - val_loss: 0.7452 - val_accuracy: 0.7525
Epoch 2/20
19/19 [==============================] - 9s 489ms/step - loss: 0.3549 - accuracy: 0.8993 - val_loss: 0.2939 - val_accuracy: 0.8997
Epoch 3/20
19/19 [==============================] - 8s 430ms/step - loss: 0.1053 - accuracy: 0.9692 - val_loss: 0.2425 - val_accuracy: 0.9264
Epoch 4/20
19/19 [==============================] - 9s 484ms/step - loss: 0.0329 - accuracy: 0.9883 - val_loss: 0.1640 - val_accuracy: 0.9431
Epoch 5/20
19/19 [==============================] - 8s 434ms/step - loss: 0.0079 - accuracy: 0.9983 - val_loss: 0.1810 - val_accuracy: 0.9498
Epoch 6/20
19/19 [==============================] - 9s 495ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.2053 - val_accuracy: 0.9532
Epoch 7/20
19/19 [==============================] - 8s 416ms/step - loss: 4.4185e-04 - accuracy: 1.0000 - val_loss: 0.2321 - val_accuracy: 0.9498
E

#### <span style="color:red">**Question 5.2.4**</span> 
**Run BiRNN for LSTM ('lstm') cell with $embed\_size= 128, state\_sizes= [64, 128], data\_manager= dm$.**

<div style="text-align: right"><span style="color:red">[1 points]</span></div>

In [152]:
bi_rnn = bi_rnn = BiRNN('lstm', embed_size=128, state_sizes=[64,128], data_manager=dm)
bi_rnn.build()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
bi_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
bi_rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

Epoch 1/20
19/19 [==============================] - 21s 739ms/step - loss: 1.3705 - accuracy: 0.5121 - val_loss: 0.8787 - val_accuracy: 0.7926
Epoch 2/20
19/19 [==============================] - 12s 653ms/step - loss: 0.4227 - accuracy: 0.8934 - val_loss: 0.5163 - val_accuracy: 0.8796
Epoch 3/20
19/19 [==============================] - 11s 595ms/step - loss: 0.1801 - accuracy: 0.9534 - val_loss: 0.2543 - val_accuracy: 0.9331
Epoch 4/20
19/19 [==============================] - 13s 660ms/step - loss: 0.0636 - accuracy: 0.9858 - val_loss: 0.1980 - val_accuracy: 0.9565
Epoch 5/20
19/19 [==============================] - 11s 591ms/step - loss: 0.0212 - accuracy: 0.9967 - val_loss: 0.1993 - val_accuracy: 0.9498
Epoch 6/20
19/19 [==============================] - 13s 658ms/step - loss: 0.0121 - accuracy: 0.9975 - val_loss: 0.8230 - val_accuracy: 0.8562
Epoch 7/20
19/19 [==============================] - 11s 596ms/step - loss: 0.0461 - accuracy: 0.9892 - val_loss: 0.1667 - val_accuracy: 0.9532

### <span style="color:#0b486b">5.3. RNNs with various types, cells, and fine-tuning embedding matrix for sequence modeling and neural embedding </span> ###

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 11 points]<span></div>

#### <span style="color:red">**Question 5.3.1**</span> 

**In what follows, you are required to combine the code in Part 1 and Part 2 to gain a general RNN which can be either Uni-directional RNN or Bi-directional RNN and the embedding matrix can be initialized using a pretrained Word2Vect.**

**Below are the descriptions of the attributes of the class *RNN*:**
- `run_mode (self.run_mode)` has three values (scratch, init-only, and init-fine-tune).
  - `scratch` means training the embedding matrix from scratch.
  - `init-only` means only initializing the embedding matrix with a pretrained Word2Vect but **not further doing** fine-tuning that matrix.
  - `init-fine-tune` means both initializing the embedding matrix with a pretrained Word2Vect and **further doing** fine-tuning that matrix.
- `network_type (self.network_type)` has two values (uni-directional and bi-directional) which correspond to either Uni-directional RNN or Bi-directional RNN.
- `cell_type (self.cell_type)` has three values (simple-rnn, gru, and lstm) which specify the memory cell used in the network.
- `embed_model (self.embed_model)` specifes the pretrained Word2Vect model used.
-  `embed_size (self.embed_size)` specifes the embedding size. Note that when run_mode is either init-only' or 'init-fine-tune', this embedding size is extracted from embed_model for dimension compatability.
- `state_sizes (self.state_sizes)` indicates the list of the hidden sizes  of memory cells. For example, $state\_sizes = [64, 128]$ means that you have two hidden layers in your network with hidden sizes of $64$ and $128$ respectively.

**Complete the code of the class *RNN*.**

<div style="text-align: right"><span style="color:red">[7 points]</span></div>

In [161]:
class RNN:
    def __init__(self, run_mode = 'scratch', cell_type= 'gru', network_type = 'uni-directional', embed_model= 'glove-wiki-gigaword-100', 
                 embed_size= 128, state_sizes = [64, 64], data_manager = None):
        self.run_mode = run_mode
        self.data_manager = data_manager
        self.cell_type = cell_type
        self.network_type = network_type
        self.state_sizes = state_sizes
        self.embed_model = embed_model
        self.embed_size = embed_size
        if self.run_mode != 'scratch':
            self.embed_size = int(self.embed_model.split("-")[-1])
        self.data_manager = data_manager
        self.vocab_size = dm.vocab_size +1
        self.word2idx = dm.word2idx
        self.word2vect = None
        self.embed_matrix = np.zeros(shape= [self.vocab_size, self.embed_size])
    
    def build_embedding_matrix(self):
        self.word2vect = api.load(self.embed_model) 
        for word, idx in self.word2idx.items():
            try:
                self.embed_matrix[idx] = self.word2vect.word_vec(word) 
            except KeyError:
                pass 
    
    @staticmethod
    def get_layer(cell_type= 'gru', network_type= 'uni-directional', hidden_size= 128, return_sequences=True, activation = 'tanh'):
        if network_type == 'uni-directional':
            return UniRNN.get_layer(cell_type=cell_type, state_size=hidden_size, return_sequences=return_sequences) 
        return BiRNN.get_layer(cell_type=cell_type, state_size=hidden_size, return_sequences=return_sequences)
    
    def build(self):
        x = tf.keras.layers.Input(shape=[None]) 
        if self.run_mode == "scratch":
            self.embedding_layer = tf.keras.layers.Embedding(self.vocab_size, self.embed_size, mask_zero=True, trainable=True)
        elif self.run_mode == "init-only":
            self.build_embedding_matrix()
            self.embedding_layer = tf.keras.layers.Embedding(self.vocab_size, self.embed_size, mask_zero=True, trainable=False, weights=[self.embed_matrix])
        elif self.run_model == "init-fine-tune": 
            self.build_embedding_matrix()
            self.embedding_layer = tf.keras.layers.Embedding(self.vocab_size, self.embed_size, mask_zero=True, trainable=True, weights=[self.embed_matrix]) 
            
        h = self.embedding_layer(x)
        num_layers=len(self.state_sizes)
        for i in range(num_layers):
            if i == num_layers - 1:
                h = self.get_layer(cell_type=self.cell_type, hidden_size=self.state_sizes[i], return_sequences=False)(h)
            else:
                h = self.get_layer(cell_type=self.cell_type, hidden_size=self.state_sizes[i])(h)
        h = tf.keras.layers.Dense(dm.num_classes, activation="softmax")(h)
        self.model = tf.keras.Model(inputs=x, outputs=h)
        
    def compile_model(self, *args, **kwargs):
        self.model.compile(*args, **kwargs)
    
    def fit(self, *args, **kwargs):
        return self.model.fit(*args, **kwargs)
    
    def evaluate(self, *args, **kwargs):
        self.model.evaluate(*args, **kwargs)       


#### <span style="color:red">**Question 5.3.2**</span> 

**Design the experiment to compare three running modes. Note that you should stick with fixed values for other attributes and only vary *run_mode*. Give your comments for the results.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div>

In [164]:
def test_mode(run_mode):
    rnn = RNN(run_mode=run_mode, cell_type='basic_rnn', embed_size=128, state_sizes=[64,128], data_manager=dm)
    rnn.build()
    opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
    rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

In [165]:
scratch_rnn = test_mode('scratch')
init_rnn = test_mode('init-only')
init_fine_rnn = test_mode('init-fine-tune')

Epoch 1/20
19/19 [==============================] - 2s 70ms/step - loss: 1.0803 - accuracy: 0.6070 - val_loss: 0.4816 - val_accuracy: 0.8763
Epoch 2/20
19/19 [==============================] - 1s 68ms/step - loss: 0.2790 - accuracy: 0.9251 - val_loss: 0.2815 - val_accuracy: 0.9197
Epoch 3/20
19/19 [==============================] - 1s 64ms/step - loss: 0.1316 - accuracy: 0.9634 - val_loss: 0.3334 - val_accuracy: 0.9064
Epoch 4/20
19/19 [==============================] - 1s 69ms/step - loss: 0.1001 - accuracy: 0.9692 - val_loss: 0.1813 - val_accuracy: 0.9365
Epoch 5/20
19/19 [==============================] - 1s 64ms/step - loss: 0.0289 - accuracy: 0.9958 - val_loss: 0.2631 - val_accuracy: 0.9264
Epoch 6/20
19/19 [==============================] - 1s 67ms/step - loss: 0.0176 - accuracy: 0.9958 - val_loss: 0.0876 - val_accuracy: 0.9666
Epoch 7/20
19/19 [==============================] - 1s 66ms/step - loss: 0.0047 - accuracy: 1.0000 - val_loss: 0.0859 - val_accuracy: 0.9632
Epoch 8/20
19

C:\Users\US\AppData\Local\Temp/ipykernel_23488/156669667.py:23: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  self.embed_matrix[idx] = self.word2vect.word_vec(word)


19/19 [==============================] - 3s 74ms/step - loss: 1.5812 - accuracy: 0.3430 - val_loss: 1.0339 - val_accuracy: 0.6120
Epoch 2/20
19/19 [==============================] - 1s 63ms/step - loss: 1.0250 - accuracy: 0.6137 - val_loss: 0.7235 - val_accuracy: 0.7391
Epoch 3/20
19/19 [==============================] - 1s 70ms/step - loss: 0.6761 - accuracy: 0.7560 - val_loss: 0.7640 - val_accuracy: 0.7358
Epoch 4/20
19/19 [==============================] - 1s 61ms/step - loss: 0.4795 - accuracy: 0.8293 - val_loss: 0.3861 - val_accuracy: 0.8662
Epoch 5/20
19/19 [==============================] - 1s 61ms/step - loss: 0.2791 - accuracy: 0.9092 - val_loss: 0.2984 - val_accuracy: 0.9064
Epoch 6/20
19/19 [==============================] - 1s 55ms/step - loss: 0.1980 - accuracy: 0.9359 - val_loss: 0.2505 - val_accuracy: 0.9264
Epoch 7/20
19/19 [==============================] - 1s 59ms/step - loss: 0.1373 - accuracy: 0.9617 - val_loss: 0.2317 - val_accuracy: 0.9331
Epoch 8/20
19/19 [======

AttributeError: 'RNN' object has no attribute 'run_model'

#### <span style="color:red">**Question 5.3.3**</span> 

**Run the above general RNN with at least five parameter sets and try to obtain the best performance. You can stick with the running mode *init-fine-tune* and use grid search to tune other parameters. Record your best model which will be used in the next part.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div>

#Report your results here

Model 1 (run_mode ='init-fine-tune',...): accuracy = ...

......................................................

In [ ]:
#The run of your best model here
my_best_rnn = #Insert your code here

### <span style="color:#0b486b">5.4. RNNs with Attention for Text and Sequence Classification</span> ###

**In what follows, you are required to implement a RNN with the attention machenism for text and sequence classification. This attention mechanism is applied at the last hidden layer of our RNN. Specifically, let $h_1, h_2,...,h_{L-1}, h_L$ be the hidden states at the last hidden layer where $L$ is the sequence length. We compute the context vector $c$ as $c=\sum_{i=1}^{L}a_{i}h_{i}$ where $a_1,...,a_L$ are the allignment weights (i.e., $a_i\geq 0$ and $\sum_{i=1}^{L}a_{i}=1$).**

**The allignment weights are computed as follows:**
- $a=[a_{i}]_{i=1}^{L}=softmax([s_{i}]_{i=1}^{L})$ where $s= [s_{i}]_{i=1}^{L}$ consists of the allignment scores.
- The assigment scores $s= [s_{i}]_{i=1}^{L}$ are computed as $s=tanh(hU)V$ where $h=\left[\begin{array}{c}
h_{1}\\
h_{2}\\
...\\
h_{L-1}\\
h_{L}
\end{array}\right]\in\mathbb{R}^{L\times state\_size_{3}}$, $U\in\mathbb{R}^{state\_size_{3}\times output\_length}$, $V\in\mathbb{R}^{output\_length\times1}$, and $output\_length$ is a hyper-parameter. Note that if we consider a mini-batch, the shape of $h$ is $batch\_size \times L \times state\_size_3$ where $state\_size_3$ is the hidden size of the last hidden layer.

After having the context vector $c$, we concatenate with the last hidden state $h_L$. On top of this concatenation, we conduct the output layer with the softmax activation.

<img src="./images/attention.png" align="center" width=1200/>

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 14 points]<span></div>

#### <span style="color:red">**Question 5.4.1**</span>

**We declare the  layer `MyAttention` as a class inherited from `tf.keras.layers.Layer` to realize our attention mechanism. You are required to provide the code for this class. Note that in the `def call(self, all_states, last_state)` method, `all_states` is the collection of all hidden states and `last_state` is the last hidden state.**

<div style="text-align: right"><span style="color:red">[5 points]</span></div>

In [ ]:
class MyAttention(tf.keras.layers.Layer):
    def __init__(self, output_length= 50):
        super(MyAttention, self).__init__()
        #Inser your code here
    
    #all_states is the collection of all hidden states and last_state is the last hidden state
    def call(self, all_states, last_state):
        #Insert your code here

#### <span style="color:red">**Question 5.4.2**</span> 

**You are required to extend the class `RNN` in Section `5.3.1` to achieve the class `Attention_RNN` in which the attention mechanism mentioned above is applied at the last hidden layer.**

<div style="text-align: right"><span style="color:red">[5 points]</span></div>

In [ ]:
class Attention_RNN:
    def __init__(self, run_mode = 'scratch', cell_type= 'gru', network_type = 'uni-directional', embed_model= 'glove-wiki-gigaword-100', 
                 embed_size= 128, state_sizes = [64, 64], data_manager = None):
        self.run_mode = run_mode
        self.data_manager = data_manager
        self.cell_type = cell_type
        self.network_type = network_type
        self.state_sizes = state_sizes
        self.embed_model = embed_model
        self.embed_size = embed_size
        if self.run_mode != 'scratch':
            self.embed_size = int(self.embed_model.split("-")[-1])
        self.data_manager = data_manager
        self.vocab_size = dm.vocab_size +1
        self.word2idx = dm.word2idx
        self.word2vect = None
        self.embed_matrix = np.zeros(shape= [self.vocab_size, self.embed_size])
    
    def build_embedding_matrix(self):
        #Insert your code here
    
    @staticmethod
    def get_layer(cell_type= 'gru', network_type= 'uni-directional', hidden_size= 128, return_sequences= False, activation = 'tanh'):
        #Insert your code here
    
    def build(self):
        #Insert your code here
        
    def compile_model(self, *args, **kwargs):
        self.model.compile(*args, **kwargs)
    
    def fit(self, *args, **kwargs):
        return self.model.fit(*args, **kwargs)
    
    def evaluate(self, *args, **kwargs):
        self.model.evaluate(*args, **kwargs)       


#### <span style="color:red">**Question 5.4.3**</span> 

**Choose a common setting for standard RNN and RNN with attention and conduct experiments to compare them. A setting here means `cell_type`, `network_type`, and list of `state sizes`**.

<div style="text-align: right"><span style="color:red">[4 points]</span></div>

### <span style="color:#0b486b">5.5. Investigating the embedding vectors from the embedding matrix</span> ###

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 10 points]<span></div>

**As you know, the embedding matrix is a collection of embedding vectors, each is for one word. In this part, you will base on the cosine similarity of the embedding vectors for the words to find the top-k most relevant words for a given word.**

**Good embeddings should have words close in meaning near each other by some similarity metrics. The similarity metric we'll use is the `consine` similarity, which is defined for two vector $\mathbf{u}$ and $\mathbf{v}$ as $\cos(\mathbf{u}, \mathbf{v})=\frac{\mathbf{u} \cdot \mathbf{v}}{\left\Vert{\mathbf{u}}\right\Vert\left\Vert{\mathbf{v}}\right\Vert}$ where $\cdot$ means dot product and $\left\Vert\cdot\right\Vert$ means the $\mathcal{L}_2$ norm.**

In [ ]:
def cosine_similarity(u,v):
    return np.dot(u,v)/(np.linalg.norm(u)*np.linalg.norm(v))

#### <span style="color:red">**Question 5.5.1**</span> 

**You are required to write the code for the function `find_most_similar(word= None, k=5, model= None)`. As its name, this function returns the `top-k most relevant word` for a given word based on the cosine similarity of the embedding vectors.**

<div style="text-align: right"><span style="color:red">[5 points]</span></div>

In [ ]:
def find_most_similar(word= None, k=5, model= None):
    try:
        #Insert your code here
    except: #word not in the vocabulary
        print("Word is not in the dictionary!")

Here is the example of the above function. As you can observe, the result makes sense which demonstrates that we obtain a good model with the meaningful embedding matrix.

In [ ]:
find_most_similar(word='poland', k=10, model= my_best_rnn)

#### <span style="color:red">**Question 5.5.2**</span> 
​
**You are required to write the code for the function `plot3D_with_labels(word= None, model= None, k= 10)`. As its name, this function visualizes the `top-k most relevant word` for a given word based on the cosine similarity of the embedding vectors by using tSNE to project the embedding vectors to a 3D space.**
​

<img src="./images/3D_plots.png" align="center" width=600/>
<div style="text-align: right"><span style="color:red">[5 points]</span></div>

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import TSNE
tsne = TSNE(perplexity=30, n_components=3, init='pca', n_iter=5000)
def plot3D_with_labels(word= None, model= None, k= 10):
   #Insert yout codeh here

In [ ]:
plot3D_with_labels(word='poland', k=20, model= rnn)

--- 
<div style="text-align: center"> <span style="color:green">GOOD LUCK WITH YOUR ASSIGNMENT 2!</span> </div>
<div style="text-align: center"> <span style="color:black">END OF ASSIGNMENT</span> </div>